## Import Essential Library And Get Data Ready

In [15]:
import os
import re
import random
from PIL import Image
import torch
from tqdm.notebook import tqdm
from transformers import CLIPModel,CLIPProcessor

In [13]:
def load_split_file(file_path):
    with open(file_path,'r') as f:
        lines= f.readlines()
    samples = []
    for line in lines:
        path, label= line.strip().split()
        samples.append((path, int(label)))
    return samples

def load_prompt(file_path):
    id_to_name = {}
    with open(file_path, 'r') as f:
        for line in f:
            idx, name = line.split()[0],re.sub(r'\d+', '', line).strip()

            id_to_name[int(idx)] = name
    return id_to_name

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [14]:
dataset_root = "/content/Classification/ip102_v1.1"

images_root = os.path.join(dataset_root, "images")
train_txt = os.path.join(dataset_root, "train.txt")
val_txt = os.path.join(dataset_root, "val.txt")
test_txt = os.path.join(dataset_root, "test.txt")

train_data = load_split_file(train_txt)
test_data = load_split_file(test_txt)
val_data = load_split_file(val_txt)

## Model defenition

In [17]:
model_name = "openai/clip-vit-base-patch16"
model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/599M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

## Test Model

In [20]:
def test_model_few_shot(path, k_shots=0):
    correct = 0
    total = 0

    id_to_name = load_prompt(path)
    num_classes = len(id_to_name)

    if k_shots == 0:
        text_prompts = [f"a photo of a {name}" for name in list(id_to_name.values())]
        text_inputs = processor(text=text_prompts, return_tensors="pt", padding=True).to(device)

        with torch.no_grad():
            try:
                text_inputs = processor(text=text_prompts, return_tensors="pt", padding=True).to(device)
                text_features = model.get_text_features(**text_inputs)
            except:
                text_inputs = processor(text=text_prompts,return_tensors="pt",padding="max_length",
                                        truncation=True,max_length=processor.tokenizer.model_max_length ).to(device)
                text_features = model.get_text_features(**text_inputs)
        class_features = text_features / text_features.norm(p=2, dim=-1, keepdim=True)

    else:
        print(f"Preparing {k_shots}-shot prototypes...")
        class_features_list = [[] for _ in range(num_classes)]
        shot_counts = {i: 0 for i in range(num_classes)}
        random.shuffle(train_data)

        for image_path, true_label in train_data:
            if shot_counts[true_label] < k_shots:
                full_path = os.path.join(images_root, image_path)
                image = Image.open(full_path).convert("RGB")
                inputs = processor(images=image, return_tensors="pt").to(device)

                with torch.no_grad():
                    image_features = model.get_image_features(**inputs)
                    image_features /= image_features.norm(p=2, dim=-1, keepdim=True)
                    class_features_list[true_label].append(image_features)
                    shot_counts[true_label] += 1

            if all(count == k_shots for count in shot_counts.values()):
                break

        class_features = torch.zeros(num_classes, model.config.projection_dim).to(device)
        for i in range(num_classes):
            if class_features_list[i]:
                class_features[i] = torch.stack(class_features_list[i]).mean(dim=0).squeeze(0)
            else:
                print(f"Warning: Not enough shots collected for class {i}")
        class_features /= class_features.norm(p=2, dim=-1, keepdim=True)

    print(f"Starting evaluation with {k_shots}-shot learning...")

    for image_path, true_label in tqdm(test_data):
        full_path = os.path.join(images_root, image_path)
        image = Image.open(full_path).convert("RGB")

        inputs = processor(images=image, return_tensors="pt").to(device)

        with torch.no_grad():
            image_features = model.get_image_features(**inputs)
            image_features /= image_features.norm(p=2, dim=-1, keepdim=True)

            similarity = (image_features @ class_features.T).squeeze(0)
            pred_label = similarity.argmax().item()

            if pred_label == true_label:
                correct += 1
            total += 1

    accuracy = correct / total * 100
    print(f"Test Accuracy ({k_shots}-shot): {accuracy:.2f}%")



### 1. using simple prompts:

In [35]:
ls /content/'caption generation'

gemini-long.txt  gemini-short.txt  gpt-long.txt  gpt-short.txt  simple.txt


In [40]:
test_model_few_shot(os.path.join(dataset_root,os.path.join("/content", "caption generation", "simple.txt")), k_shots=0)
test_model_few_shot(os.path.join(dataset_root,os.path.join("/content", "caption generation", "simple.txt")), k_shots=5)
test_model_few_shot(os.path.join(dataset_root,os.path.join("/content", "caption generation", "simple.txt")), k_shots=10)

Starting evaluation with 0-shot learning...


  0%|          | 0/22619 [00:00<?, ?it/s]

Test Accuracy (0-shot): 13.58%
Preparing 5-shot prototypes...
Starting evaluation with 5-shot learning...


  0%|          | 0/22619 [00:00<?, ?it/s]

Test Accuracy (5-shot): 27.82%
Preparing 10-shot prototypes...
Starting evaluation with 10-shot learning...


  0%|          | 0/22619 [00:00<?, ?it/s]

Test Accuracy (10-shot): 32.41%


### 2. using  prompts generated by GEMINI:

In [41]:
test_model_few_shot(os.path.join(dataset_root, os.path.join("/content", "caption generation", "gemini-short.txt")), k_shots=0)
test_model_few_shot(os.path.join(dataset_root, os.path.join("/content", "caption generation", "gemini-short.txt")), k_shots=5)
test_model_few_shot(os.path.join(dataset_root, os.path.join("/content", "caption generation", "gemini-short.txt")), k_shots=10)

Starting evaluation with 0-shot learning...


  0%|          | 0/22619 [00:00<?, ?it/s]

Test Accuracy (0-shot): 23.12%
Preparing 5-shot prototypes...
Starting evaluation with 5-shot learning...


  0%|          | 0/22619 [00:00<?, ?it/s]

Test Accuracy (5-shot): 27.80%
Preparing 10-shot prototypes...
Starting evaluation with 10-shot learning...


  0%|          | 0/22619 [00:00<?, ?it/s]

Test Accuracy (10-shot): 33.92%


In [42]:
test_model_few_shot(os.path.join(dataset_root, os.path.join("/content", "caption generation", "gemini-long.txt")), k_shots=0)
test_model_few_shot(os.path.join(dataset_root, os.path.join("/content", "caption generation", "gemini-long.txt")), k_shots=5)
test_model_few_shot(os.path.join(dataset_root, os.path.join("/content", "caption generation", "gemini-long.txt")), k_shots=10)

Token indices sequence length is longer than the specified maximum sequence length for this model (78 > 77). Running this sequence through the model will result in indexing errors


Starting evaluation with 0-shot learning...


  0%|          | 0/22619 [00:00<?, ?it/s]

Test Accuracy (0-shot): 22.64%
Preparing 5-shot prototypes...
Starting evaluation with 5-shot learning...


  0%|          | 0/22619 [00:00<?, ?it/s]

Test Accuracy (5-shot): 28.72%
Preparing 10-shot prototypes...
Starting evaluation with 10-shot learning...


  0%|          | 0/22619 [00:00<?, ?it/s]

Test Accuracy (10-shot): 33.52%


### 3. using  prompts generated by GPT:

In [43]:
test_model_few_shot(os.path.join(dataset_root, os.path.join("/content", "caption generation", "gpt-short.txt")), k_shots=0)
test_model_few_shot(os.path.join(dataset_root, os.path.join("/content", "caption generation", "gpt-short.txt")), k_shots=5)
test_model_few_shot(os.path.join(dataset_root, os.path.join("/content", "caption generation", "gpt-short.txt")), k_shots=10)

Starting evaluation with 0-shot learning...


  0%|          | 0/22619 [00:00<?, ?it/s]

Test Accuracy (0-shot): 20.93%
Preparing 5-shot prototypes...
Starting evaluation with 5-shot learning...


  0%|          | 0/22619 [00:00<?, ?it/s]

Test Accuracy (5-shot): 27.19%
Preparing 10-shot prototypes...
Starting evaluation with 10-shot learning...


  0%|          | 0/22619 [00:00<?, ?it/s]

Test Accuracy (10-shot): 33.86%


In [44]:
test_model_few_shot(os.path.join(dataset_root, os.path.join("/content", "caption generation", "gpt-long.txt")), k_shots=0)
test_model_few_shot(os.path.join(dataset_root, os.path.join("/content", "caption generation", "gpt-long.txt")), k_shots=5)
test_model_few_shot(os.path.join(dataset_root, os.path.join("/content", "caption generation", "gpt-long.txt")), k_shots=10)

Starting evaluation with 0-shot learning...


  0%|          | 0/22619 [00:00<?, ?it/s]

Test Accuracy (0-shot): 17.98%
Preparing 5-shot prototypes...
Starting evaluation with 5-shot learning...


  0%|          | 0/22619 [00:00<?, ?it/s]

Test Accuracy (5-shot): 27.28%
Preparing 10-shot prototypes...
Starting evaluation with 10-shot learning...


  0%|          | 0/22619 [00:00<?, ?it/s]

Test Accuracy (10-shot): 34.17%
